## Aufgabe: 

Trainiere ein Modell, welches Katzen und Hunden unterscheiden kann!

### Lösungstipp 1:

Dieses Modell wird recht komplex werden, und entsprechend lange zum Training benötigen. Später werden wir noch ein schnelleres Modell kennenlernen - für diese Aufgabe möchtest du aber auf jeden Fall eine GPU zur Verfügung haben. 

### Lösungstipp 2:

Ggf. macht es Sinn, die Datenmenge erstmal zu reduzieren - beispielsweise könntest du mit 1000 Hund-Bildern und 1000 Katzen-Bildern starten. Du darfst den Code aus den letzten Lektionen 1:1 übernehmen, anpassen,...

### Lösungstipp 3:

Um die Bilder korrekt zu klassifizieren, wirst du u.U. mehrere Layer benötigen. Du könntest hier z.B. einen Aufbau ähnlich zum VGG-Netz verwenden, also mehrere Blöcke mit jeweils 2-3 Convolutional-Layern, die von einem MaxPooling abgeschlossen werden. Nach mehreren dieser Blöcke kannst du dann 2-3 Dense-Layer platzieren, um die Ergebnisse auf 1 Neuron herunterzubrechen.

### Lösungstipp 4:

Ggf. sind die y-Daten noch nicht im richtigen Format. Du brauchst hier eine Shape von (X, 1), wobei X hier die Anzahl der Datensätze sind. Die Shape der y-Variable kannst du dir über `y.shape` ausgeben, sollte die Shape nicht `(X, 1)` sein, so kannst du hierfür den `.reshape`-Befehl verwenden.

### Lösungstipp 5: 

Gegebenenfalls wird dein Netz trotzdem die Daten nicht korrekt lernen können. Das liegt dann daran, dass die Lernrate zu hoch ist - in dem Fall musst du die Lernrate verringern. Beispiel (für "adam"):

```python
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.000001), loss="binary_crossentropy", metrics=["acc"])
```


## Herunterladen der Daten

In [ ]:
import requests
import numpy as np
import math
import os
import zipfile
from tqdm import tqdm_notebook as tqdm 

if not os.path.exists("data"):
    os.mkdir("data")
    
if not os.path.exists(os.path.join("data", "PetImages")):
    url = "https://downloads.codingcoursestv.eu/037%20-%20neuronale%20netze/PetImages.zip"
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0)); 
    block_size = 1024
    
    print("Downloading...")
    with open(os.path.join("data", "PetImages.zip"), 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size), unit='KB', unit_divisor=1024, unit_scale=True):
            f.write(data)
            
    print("Download completed")
    print("Extracting...")
    
    zip_ref = zipfile.ZipFile(os.path.join("data", "PetImages.zip"), 'r')
    zip_ref.extractall(os.path.join("data"))
    zip_ref.close()
    
    print("Done!")
else:
    print("Die Dateien existieren bereits")


Die Dateien existieren bereits


## Einlesen der Daten

In [ ]:
import numpy as np
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    # Limit to 1000 files for faster processing
    files = files[:1000]
    
    images = []
    # Loop over all files in folder with tqdm to show progress
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
            image = image.resize((224, 224), Image.LANCZOS)

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
            image = image.convert("RGB")
            # Convert to numpy array
            image = np.asarray(image)
            # Append to list
            images.append(image)
        except OSError:
            pass
    
    return images
    
cats = read_images(os.path.join("data", "PetImages", "Cat"))
dogs = read_images(os.path.join("data", "PetImages", "Dog"))

/var/folders/by/l34wfpgn2v11pqz8wlmb4v080000gn/T/ipykernel_27332/2638750310.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/Users/alex/anaconda3/envs/ml/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


## Preprocessing der Bilddaten von Dogs and Cats

### Convert to numpy arrays

In [3]:
dogs = np.array(dogs)
cats = np.array(cats)

# Combine dogs and cats to one dataset
X = np.concatenate((dogs, cats), axis=0).astype(np.float32) / 255.
print("X shape:", X.shape)

X shape: (2000, 224, 224, 3)


### Create labels

Dogs = 0 and Cats = 1


In [4]:
# Dogs = 0, Cats = 1
y_dogs = np.zeros(len(dogs))
y_cats = np.ones(len(cats))

# Combine labels
y = np.concatenate((y_dogs, y_cats), axis=0)
y = y.reshape(-1, 1).astype(np.float32)
print("y shape:", y.shape)

y shape: (2000, 1)


## Definition des Modells

In Anlehnung an das VGG16-Modell

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    51,381,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,684,833 (200.98 MB)

 Trainable params: 52,684,833 (200.98 MB)

 Non-trainable params: 0 (0.00 B)

## Model Training

In [9]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.6250 - loss: 0.6562 - val_accuracy: 0.0000e+00 - val_loss: 0.9346
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.6250 - loss: 0.6527 - val_accuracy: 0.0000e+00 - val_loss: 0.8701
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.6250 - loss: 0.6456 - val_accuracy: 0.0000e+00 - val_loss: 1.0162
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.6288 - loss: 0.6424 - val_accuracy: 0.0000e+00 - val_loss: 0.9668
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.6294 - loss: 0.6271 - val_accuracy: 0.0525 - val_loss: 0.8430
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - accuracy: 0.6594 - loss: 0.6016 - val_accuracy: 0.4050 - val_loss: 0.7700
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.7031 - loss: 0.5722 - val_accuracy: 0.5875 - val_loss: 0.6843
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.7175 - loss: 0.5430 - val_accuracy:

## Abspeichern des trainierten Modells

In [10]:
# Abspeichern des trainierten Modells
model.save(os.path.join("data", "dogs_vs_cats_model.keras"))